## Question 1

$$X = [X_1, X_2] \ and \ \beta = \begin{bmatrix} \beta_1 \\ \beta_2 \end{bmatrix}$$
$$\hat\beta =  (X'X)^{-1}X'y$$

**Solving**
$$ y = X \beta + e$$
$$ y = [X_1, X_2] \begin{bmatrix} \beta_1 \\ \beta_2 \end{bmatrix} + e $$
$$ y = X_1 \beta_1 + X_2 \beta_2 + e $$
$$Multiplying \ above \ equation \ with \ X_1' \ and \ X_2' \ on \ both \ sides$$
$$ X_1'y = X_1'X_1 \beta_1 + X_1'X_2 \beta_2 + X_1'e $$
$$ X_2'y = X_2'X_1 \beta_1 + X_2'X_2 \beta_2 + X_2'e $$
$$ Here \ X_1'e \ and \ X_2'e = 0 $$ 
So, 
$$ X_1'y = X_1'X_1 \beta_1 + X_1'X_2 \beta_2 \ --eq. (2)$$
$$ X_2'y = X_2'X_1 \beta_1 + X_2'X_2 \beta_2 \ --eq. (3)$$

$$ Solving \ for \ \beta_1$$
$$ \hat\beta_1 = X_1'(y - X_2\beta_2)(X_1'X_1)^{-1}$$

$$ Multiplying \ eq. (2) \ by \ X_2'X_1(X_1'X_1)^{-1} $$ 

$$ X_2'X_1\beta_1 + X_2'X_1(X_1'X_1)^{-1}X_1'X_2\beta_2 = X_2'X_1(X_1'X_1)^{-1}X_1'y $$

$$ [X_2'X_2 - X_2'X_1(X_1'X_1)^{-1}X_1'X_2] \beta_2 = X_2'y - X_2'X_1(X_1'X_1)^{-1}X_1'y $$
$$ Let \ P_1 = X_1(X_1'X_1)^{-1}X_1' $$
$$ [X_2'X_2 - X_2'P_1X_2] \beta_2 = X_2'y - X_2'P_1y $$
$$ [X_2'(I-P_1)X_2] \beta_2 = X_2'(I-P_1)y $$

Therefore,
$$  \hat\beta_2 = [X_2'(I-P_1)X_2]^{-1} X_2'(I-P_1)y $$
$$ \hat\beta_1 = X_1'(y - X_2\beta_2)(X_1'X_1)^{-1}$$

## Question 2

In [29]:
import numpy as np
import pandas as pd

In [30]:
df = pd.read_csv("C:/Users/shaur/Desktop/UTD/PDS/Assignments/BWGHT.csv")
df.head()

,faminc,cigtax,cigprice,bwght,fatheduc,motheduc,parity,male,white,cigs,lbwght,bwghtlbs,packs,lfaminc
0,13.5,16.5,122.3,109,12.0,12.0,1,1,1,0,4.691348,6.8125,0.0,2.602690
1,7.5,16.5,122.3,133,6.0,12.0,2,1,0,0,4.890349,8.3125,0.0,2.014903
2,0.5,16.5,122.3,129,NaN,12.0,2,0,0,0,4.859812,8.0625,0.0,-0.693147
3,15.5,16.5,122.3,126,12.0,12.0,2,1,0,0,4.836282,7.8750,0.0,2.740840
4,27.5,16.5,122.3,134,14.0,12.0,2,1,1,0,4.897840,8.3750,0.0,3.314186


In [31]:
df['(intercept)'] = 1
x = df[['(intercept)','cigs','faminc']]
y = df['bwght']
b = np.linalg.solve(x.T@x,x.T@y)
e = y-x@b
vb = e.var()*np.linalg.inv(x.T@x)
se = np.sqrt(np.diagonal(vb))
tstat = b/se
tstat

array([111.5923058 ,  -5.06396699,   3.18048888])

In [33]:
import scipy.stats as sps

class linear_model:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.b = np.linalg.solve(x.T@x,x.T@y)
        e = y-x@self.b
        self.vb = self.vcov_b(e)
        self.se = np.sqrt(np.diagonal(self.vb))
        self.t = self.b/self.se
        self.p = 2*sps.norm.cdf(-np.abs(self.t))
        self.rsq = 1-e.var()/y.var()
    def vcov_b(self,e):
        x = self.x
        return e.var()*np.linalg.inv(x.T@x)

class white(linear_model):
    def vcov_b(self,e):
        x = self.x
        meat = np.diagflat(e.values**2)
        bread = np.linalg.solve(x.T@x,x.T)
        sandwich = bread@meat@bread.T
        return sandwich
        
class newey(linear_model):
    def omega(self):
        x = self.x
        m_0 = np.diagflat([np.average(e.values**2)]*len(e))
        m_n1 = np.zeros(len(e)-1)
        for i in range(0, len(e)-1):
            m_n1[i] = e[i]*e[i+1]
        
        m_1 = np.diagflat([np.sum(m_n1)/len(e)]*(len(e)-1), 1)
        m_2 = np.diagflat([np.sum(m_n1)/len(e)]*(len(e)-1), -1)
        m_n = m_2 + m_0 + m_1
        k = np.linalg.inv(x.T@x)
        var_k = k@x.T@m_n@x@k
        se_nw = np.sqrt(np.diagonal(var_k))
        tstat_nw = self.b/se_nw
        return tstat_nw

In [34]:
newey(x,y).omega()

array([110.30765407,  -5.05943423,   3.1753672 ])